In [197]:
import requests
import os

from dotenv import load_dotenv
load_dotenv()

base_url = os.environ["SERVICENOW_BASEURL"]
user = os.environ["SERVICENOW_USERNAME"]
pwd = os.environ["SERVICENOW_PASSWORD"]
headers = {"Content-Type":"application/json","Accept":"application/json"}


def get_specific_catalog_item(sys_id):

    url = f'{base_url}/sn_sc/servicecatalog/items/{sys_id}'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    data = response.json()
    return data




def get_table_response(reference):
    url = f'{base_url}/now/table/{reference}?sysparm_limit=1'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    data = response.json()
    return data


def get_table_sysid(reference, table_field, value_for_query):
    if('@' in value_for_query):
        value_for_query = value_for_query.replace('@','%40')
    sysparm_query = f'{table_field}%3D{value_for_query}' 

    url = f'{base_url}/now/table/{reference}?sysparm_query={sysparm_query}&sysparm_limit=1'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()

    data = response.json()
    return data


def get_catalog_item_variables(sys_id):
    url = f'{base_url}/sn_sc/servicecatalog/items/{sys_id}/variables'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
    print(response)
    data = response.json()
    return data

def get_table_values(reference):
    url = f'{base_url}/now/table/{reference}'
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    data = response.json()
    return data


In [198]:
from langchain.schema import HumanMessage
import json
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

def create_custom_function(variable_info, custom_functions):
    inner_json = {}
    inner_json['type'] = 'string'
    if 'refenece_values' in variable_info:
        inner_json['description'] = f"Please select one of the following options for the {variable_info['name']}: {variable_info['refenece_values']}."
    else:
        inner_json['description'] = f"If {variable_info['name']} not mentioned in user query then assign {variable_info['displayvalue']}."
    custom_functions[0]['parameters']['properties'][variable_info['name']] = inner_json
    return custom_functions


def function_calling_catVar(catalog_variables):
    custom_functions = [
        {
            'name': 'extract_catalog_variables',
            'description': 'Get the user query from the body of the input text',
            'parameters': {
                'type': 'object',
                'properties': {}
            }
        }
        ]

    for variable_info in catalog_variables:
        if 'children' in variable_info:
            for child_var_info in variable_info['children']:
                # print(child_var_info)
                custom_functions = create_custom_function(child_var_info, custom_functions)
        else:
            custom_functions = create_custom_function(variable_info, custom_functions)
    return custom_functions


def get_all_variables_List(catalog_item_variable):
    def extract_variables(variables):
        variable_List = []
        for variable_info in variables:
            if variable_info['type'] == 24:
                continue
            if 'children' in variable_info:
                variable_List.extend(extract_variables(variable_info['children']))
            else:
                variable_List.append(variable_info)
        return variable_List
    
    return extract_variables(catalog_item_variable)

def set_reference_value(variable):
    if(variable['type'] == 8 and variable['dynamic_value_field'] == ''):
        reference = variable['reference']
        table_content = get_table_values(reference=reference)
        names_in_table = []
        for content in table_content['result']:
            if('u_name' in content):
                names_in_table.append(content['u_name'])
        variable['refenece_values'] = ", ".join(names_in_table)
        return variable
    
    elif (variable['type'] == 5 or variable['type'] == 18):
        choiceList = []
        for choice in variable['choices']:
            choiceList.append(choice['value'])
        variable['refenece_values'] = ", ".join(choiceList)
        return variable
    else:
        return variable


def mandatory_var_not_added(catalog_variables, json_info):

    mandatory_variable_not_added = []
    for variable_info in catalog_variables:
        if not json_info.get(variable_info['name'], ""):
            if(variable_info["mandatory"] == True):
                # print(variable_info['name'])
                #if(variable_info["read_only"] == False):
                mandatory_variable_not_added.append(variable_info['name'])
    return mandatory_variable_not_added
                

def get_valuefor_reqfor(variable_info, missing_variables, existing_variable):
        reference = variable_info['reference']
        sysparm_for_query = 'user_name'
        value_for_query = os.environ["SERVICENOW_USERNAME"]
        # print(reference + sysparm_for_query + value_for_query)
        sys_id = get_table_sysid(reference, sysparm_for_query, value_for_query)['result'][0]['sys_id']
        existing_variable[variable_info['name']] = sys_id
        if(variable_info['name'] in missing_variables):
            missing_variables.remove(variable_info['name'])
        return missing_variables, existing_variable


def get_valuefor_reference(variable_info, missing_variables, existing_variable):
    reference = variable_info['reference'] # u_software

    sample_table_response = get_table_response(reference)
    if 'u_name' in sample_table_response['result'][0]:
        sysparm_for_query = 'u_name' # name
    elif 'name' in sample_table_response['result'][0]:
        sysparm_for_query = 'name'
    
    # print(f"type 8 = {variable_info['name']}")
    if(variable_info['name'] in existing_variable):
        # print(f"type 8 = {variable_info['name']}")
        value_for_query = existing_variable[variable_info['name']] # python
        print(reference + sysparm_for_query + value_for_query)
        sys_id = get_table_sysid(reference, sysparm_for_query, value_for_query)['result'][0]['sys_id']
        existing_variable[variable_info['name']] = sys_id
    return missing_variables, existing_variable


def get_valuefor_dynamicvalues(catalog_variables, variable_info, missing_variables, existing_variable):
    for variables in catalog_variables:
        if(variables['id'] == variable_info['dynamic_value_field']):
            reference = variables['reference']
            sysparm_for_query = 'sys_id'# variable_info['dynamic_value_dot_walk_path']
            value_for_query = existing_variable[variables['name']]
            # print(" reference =  "+ reference +" sysparm_for_query =  "+ sysparm_for_query +" value_for_query = "+ value_for_query)
            requireddetails = get_table_sysid(reference, sysparm_for_query, value_for_query)['result'][0][variable_info['dynamic_value_dot_walk_path']]
            finalDetail = ''
            print(requireddetails)
            if('value' in requireddetails):
                finalDetail = requireddetails['value']
            else:
                finalDetail = requireddetails
            
            existing_variable[variable_info['name']] = finalDetail
            if(variable_info['name'] in missing_variables):
                missing_variables.remove(variable_info['name'])
    return missing_variables, existing_variable




def assign_complex_variables(prepared_api_request, catalog_variables, variables_List):
    existing_variable = prepared_api_request['existing_variables']
    missing_variables = prepared_api_request['missing_variables']
    for variable_info in variables_List:
        # print(variable_info['name'])
        if(variable_info['type'] == 31):            # for requested_for type
            missing_variables, existing_variable = get_valuefor_reqfor(variable_info, missing_variables, existing_variable)

        elif(variable_info['name'] in existing_variable and variable_info['type'] == 8):
            missing_variables, existing_variable = get_valuefor_reference(variable_info, missing_variables, existing_variable)

        elif (variable_info['dynamic_value_field'] != ''):
            missing_variables, existing_variable = get_valuefor_dynamicvalues(catalog_variables, variable_info, missing_variables, existing_variable)

    return missing_variables, existing_variable

def fetch_ui_actions(action_field, catalog_variables):
    ui_action_variable_list = []
    for action_variables in action_field:
        action_variable_id = action_variables['name'].split(':')[1] 
        for variable in catalog_variables:
            # print(f"{variable['id']} = {action_variable_id}")
            if(variable['id'] == action_variable_id):
                ui_action_variable_list.append({'name': variable['name'], 'mandatory': action_variables['mandatory']})
    
    return ui_action_variable_list


def fetch_ui_policy(catalog_item, catalog_variables):
    final_ui_condition_action = []
    for catalog_ui_policy in catalog_item['result']['ui_policy']:
        for condition_field in catalog_ui_policy['conditions']:
            action_field = catalog_ui_policy['actions']
            ui_action_variable_list = fetch_ui_actions(action_field, catalog_variables)
            variable_id = condition_field['field'].split(':')[1]
            for variable in catalog_variables:
                if variable['id'] == variable_id:
                    ui_condition = {'condition_variable_name':variable['name'],'condition_variable_value': condition_field['value'], 'condition_operation': condition_field['oper'], 'ui_actions': ui_action_variable_list}
                    final_ui_condition_action.append(ui_condition)
    return final_ui_condition_action

def arrange_ui_policy(ui_policy,missing_variables,existing_variables):
    # print(ui_policy)
    mandatory_variables_basedon_ui = []
    for ui_policy in ui_policy:
        if(ui_policy["condition_variable_name"] in existing_variables):
            # print(ui_policy["condition_variable_name"])
            if(ui_policy['condition_operation'] == '='):
                if(existing_variables[ui_policy["condition_variable_name"]] == ui_policy['condition_variable_value']):
                    for action in ui_policy['ui_actions']:
                        if(action['mandatory'] == 'true'):
                            if(action['name'] in existing_variables):
                                continue
                            elif (action['name'] not in missing_variables):
                                mandatory_variables_basedon_ui.append(action['name'])   
                                
        elif(ui_policy["condition_variable_name"] in missing_variables):
            continue
        else:
            mandatory_variables_basedon_ui.append(ui_policy["condition_variable_name"])
    return mandatory_variables_basedon_ui


def get_variable_from_query(user_query, sys_id):
    catalog_item = get_specific_catalog_item(sys_id)
    catalog_variables = catalog_item['result']['variables']
    # print(catalog_item)
    ui_policy = fetch_ui_policy(catalog_item, get_all_variables_List(catalog_variables))
    # print(ui_policy)
    variables_List = []
    for variable in get_all_variables_List(catalog_variables):
        variables_List.append(set_reference_value(variable))
    # print(variables_List)
    custom_function = function_calling_catVar(catalog_variables)
    # print(custom_function)
    message = model.predict_messages(
        [HumanMessage(content=user_query)],
        functions = custom_function
    )
    # print(message)
    print(json.dumps(message.__dict__))

    if message.additional_kwargs != {}:
        print("The 'additional_kwargs' attribute exists.")

        parse_variable_details = json.loads(message.additional_kwargs["function_call"]["arguments"])

        mandatory_variable_not_added = mandatory_var_not_added(variables_List, parse_variable_details)
        response = {"existing_variables": parse_variable_details, "missing_variables": mandatory_variable_not_added}
        print(response)
        missing_variables, existing_variables = assign_complex_variables(response, catalog_variables, variables_List)
        
        mandatory_variables_basedon_ui = arrange_ui_policy(ui_policy, missing_variables, existing_variables)

        # missing_variables, existing_variables = assign_complex_variables({'existing_variables': existing_variables, 'missing_variables': missing_variables}, catalog_variables, variables_List)

        final_result = {'existing_variables': existing_variables, 'missing_variables': missing_variables, 'ui_mandatory_variable': mandatory_variables_basedon_ui}

        print(final_result)

    else:
        final_result = {}
    return final_result    

In [200]:
get_variable_from_query("I need to install python", "50572ffac3405a1068d8b132b4013177")

{"content": "", "additional_kwargs": {"function_call": {"arguments": "{\"business_service\":\"Software Installation\",\"software_name\":\"Python\"}", "name": "extract_catalog_variables"}}, "response_metadata": {"token_usage": {"completion_tokens": 22, "prompt_tokens": 2874, "total_tokens": 2896}, "model_name": "gpt-4o-2024-05-13", "system_fingerprint": "fp_abc28019ad", "prompt_filter_results": [{"prompt_index": 0, "content_filter_results": {"hate": {"filtered": false, "severity": "safe"}, "self_harm": {"filtered": false, "severity": "safe"}, "sexual": {"filtered": false, "severity": "safe"}, "violence": {"filtered": false, "severity": "safe"}}}], "finish_reason": "function_call", "logprobs": null, "content_filter_results": {}}, "type": "ai", "name": null, "id": "run-6c36dce3-50c3-4d07-a1cc-17ad5c08bde1-0", "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": {"input_tokens": 2874, "output_tokens": 22, "total_tokens": 2896}}
The 'additional_kwargs' attribute e

{'existing_variables': {'business_service': 'effae772c3805a1068d8b132b40131dd',
  'software_name': '7c29b2afc3441a5068d8b132b4013164',
  'requested_for': '6816f79cc0a8016401c5a33be04be441',
  'Location': '163c49b037d0200044e0bfc8bcbe5dcc',
  'project_name': 'a581ab703710200044e0bfc8bcbe5de8'},
 'missing_variables': ['contact_number', 'select_your_host_name'],
 'ui_mandatory_variable': ['request_type',
  'duration',
  'add_software',
  'add_software1']}